# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

Matplotlib is building the font cache; this may take a moment.


<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298405421177                   -0.85    5.1
  2   -8.300210790523       -2.74       -1.26    1.2
  3   -8.300448140198       -3.62       -1.89    3.5
  4   -8.300462449433       -4.84       -2.77    1.2
  5   -8.300464411029       -5.71       -3.11    5.2
  6   -8.300464524980       -6.94       -3.27    7.1
  7   -8.300464581874       -7.24       -3.41    2.4
  8   -8.300464613748       -7.50       -3.54    1.0
  9   -8.300464635207       -7.67       -3.72    1.0
 10   -8.300464640784       -8.25       -3.88    1.0
 11   -8.300464643738       -8.53       -4.25    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64187415234                   -0.70    5.5
  2   -16.64603026081       -2.38       -1.14    1.0
  3   -16.67921985126       -1.48       -1.88    3.8
  4   -16.67927119941       -4.29       -2.65    4.8
  5   -16.67928600236       -4.83       -3.03    4.4
  6   -16.67928619047       -6.73       -3.36    1.6
  7   -16.67928621872       -7.55       -3.88    1.0
  8   -16.67928622409       -8.27       -4.60    1.6


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32725058460                   -0.56    7.2
  2   -33.33449889098       -2.14       -1.00    1.2
  3   -33.33603375661       -2.81       -1.72    5.5
  4   -33.33685248131       -3.09       -2.32    7.1
  5   -33.33693748856       -4.07       -2.89    7.5
  6   -33.33694389241       -5.19       -3.82    3.6
  7   -33.33694391699       -7.61       -4.20    4.5


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298441484624                   -0.85    5.0
  2   -8.300268986530       -2.74       -1.59    1.0
  3   -8.300424843442       -3.81       -2.40    5.8
  4   -8.300319782709   +   -3.98       -2.17    6.4
  5   -8.300464485856       -3.84       -3.68    3.1
  6   -8.300464593504       -6.97       -3.94    2.4
  7   -8.300464636336       -7.37       -4.34    1.8


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32538832729                   -0.56    6.0
  2   -33.32119982883   +   -2.38       -1.27    1.0
  3   -19.84734698998   +    1.13       -0.50    8.5
  4   -33.30425234311        1.13       -1.59    8.2
  5   -33.24488257519   +   -1.23       -1.34    3.1
  6   -32.89505531299   +   -0.46       -1.24    4.1
  7   -33.29778090377       -0.39       -1.75    4.8
  8   -33.33659337833       -1.41       -2.39    3.2
  9   -33.33637401180   +   -3.66       -2.52    3.2
 10   -33.33659954165       -3.65       -2.62    2.2
 11   -33.33689072083       -3.54       -3.14    2.1
 12   -33.33694259375       -4.29       -3.57    3.5
 13   -33.33694316238       -6.25       -3.73    2.9
 14   -33.33694379447       -6.20       -3.98    2.5
 15   -33.33694389365       -7.00       -4.49    3.0


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.